**Goal**: Transform transactional data into customer-level RFM and behavioral features suitable for churn modeling while preventing data leakage.

In [2]:
import pandas as pd
import numpy as np

# Load cleaned data
df = pd.read_csv(
    "../data/processed/clean_transactions.csv",
    parse_dates=["InvoiceDate"]
)

# Load churn labels
churn_labels = pd.read_csv("../data/processed/churn_labels.csv")


In [3]:
#Redefining time logic here
analysis_date = df["InvoiceDate"].max() - pd.Timedelta(days=30)

OBSERVATION_WINDOW = 90
PREDICTION_WINDOW = 30

observation_start = analysis_date - pd.Timedelta(days=OBSERVATION_WINDOW)

obs_df = df[
    (df["InvoiceDate"] >= observation_start) &
    (df["InvoiceDate"] <= analysis_date)
]


In [4]:
#Work only on observation window data
obs_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
209260,563018,16169E,WRAP 50'S CHRISTMAS,25,2011-08-11 12:53:00,0.42,13427,United Kingdom
209261,563018,23263,SET OF 3 WOODEN HEART DECORATIONS,12,2011-08-11 12:53:00,1.25,13427,United Kingdom
209262,563018,23546,WRAP PAISLEY PARK,25,2011-08-11 12:53:00,0.42,13427,United Kingdom
209263,563018,23547,WRAP FLOWER SHOP,25,2011-08-11 12:53:00,0.42,13427,United Kingdom
209264,563018,22178,VICTORIAN GLASS HANGING T-LIGHT,24,2011-08-11 12:53:00,1.25,13427,United Kingdom


In [5]:
#Sanity check
obs_df["InvoiceDate"].min(), obs_df["InvoiceDate"].max()

(Timestamp('2011-08-11 12:53:00'), Timestamp('2011-11-09 12:49:00'))

In [6]:
#Creating a reference date for RFM
#Using the same analysis_date (critical for consistency)
reference_date = analysis_date

In [7]:
#Building RFM features
#Recency (days since last purchase)
recency = (
    obs_df
    .groupby("CustomerID")["InvoiceDate"]
    .max()
    .reset_index()
)

recency["recency_days"] = (reference_date - recency["InvoiceDate"]).dt.days
recency = recency[["CustomerID", "recency_days"]]


In [8]:
#Frequency (number of transactions)
frequency = (
    obs_df
    .groupby("CustomerID")
    .size()
    .reset_index(name="txn_count_90d")
)

In [ ]:
#Monetary (spend)
obs_df["total_price"] = obs_df["Quantity"] * obs_df["UnitPrice"]

monetary = (
    obs_df
    .groupby("CustomerID")["total_price"]
    .sum()
    .reset_index(name="spend_90d")
)

In [10]:
#Behavioral retail features
#Average basket value
avg_basket = (
    obs_df
    .groupby(["CustomerID", "InvoiceNo"])["total_price"]
    .sum()
    .reset_index()
    .groupby("CustomerID")["total_price"]
    .mean()
    .reset_index(name="avg_basket_value")
)


In [11]:
#Product diversity (engagement proxy)
product_diversity = (
    obs_df
    .groupby("CustomerID")["StockCode"]
    .nunique()
    .reset_index(name="unique_products_90d")
)


In [12]:
#Country consistency
country_count = (
    obs_df
    .groupby("CustomerID")["Country"]
    .nunique()
    .reset_index(name="country_count")
)

In [13]:
#Merging all features into ONE table
features = recency \
    .merge(frequency, on="CustomerID") \
    .merge(monetary, on="CustomerID") \
    .merge(avg_basket, on="CustomerID") \
    .merge(product_diversity, on="CustomerID") \
    .merge(country_count, on="CustomerID")

In [ ]:
features.head()

,CustomerID,recency_days,txn_count_90d,spend_90d,avg_basket_value,unique_products_90d,country_count
0,12347,9,47,1294.32,1294.320000,47,1
1,12348,44,3,310.00,310.000000,3,1
2,12352,5,47,944.23,314.743333,39,1
3,12357,2,131,6207.67,6207.670000,131,1
4,12359,27,105,2876.85,2876.850000,100,1


In [15]:
features.shape

(2570, 7)

In [16]:
#Joining churn labels
model_df = features.merge(churn_labels, on="CustomerID")
model_df.head()

,CustomerID,recency_days,txn_count_90d,spend_90d,avg_basket_value,unique_products_90d,country_count,churn
0,12347,9,47,1294.32,1294.320000,47,1,0
1,12348,44,3,310.00,310.000000,3,1,1
2,12352,5,47,944.23,314.743333,39,1,1
3,12357,2,131,6207.67,6207.670000,131,1,1
4,12359,27,105,2876.85,2876.850000,100,1,1


In [17]:
#Sanity checks
model_df.isnull().sum()

CustomerID             0
recency_days           0
txn_count_90d          0
spend_90d              0
avg_basket_value       0
unique_products_90d    0
country_count          0
churn                  0
dtype: int64

In [18]:
model_df["churn"].value_counts(normalize=True)

churn
1    0.568434
0    0.431566
Name: proportion, dtype: float64

In [19]:
#Light feature validation
model_df.describe()

,CustomerID,recency_days,txn_count_90d,spend_90d,avg_basket_value,unique_products_90d,country_count,churn
count,2528.000000,2528.000000,2528.000000,2528.000000,2528.000000,2528.000000,2528.000000,2528.000000
mean,15260.046677,29.276503,48.658228,1075.456765,443.473509,40.458070,1.000791,0.568434
std,1697.845988,23.326791,101.793370,4313.248767,508.211593,53.149882,0.028122,0.495393
min,12347.000000,0.000000,2.000000,6.200000,3.450000,1.000000,1.000000,0.000000
25%,13811.500000,9.000000,14.000000,266.992500,202.672500,14.000000,1.000000,0.000000
50%,15261.500000,25.000000,29.000000,477.835000,321.436667,27.000000,1.000000,1.000000
75%,16710.500000,44.000000,56.000000,962.185000,500.709500,50.000000,1.000000,1.000000
max,18287.000000,89.000000,2688.000000,117348.450000,7271.340000,1031.000000,2.000000,1.000000


In [20]:
#Saving modeling dataset (ML-ready)
model_df.to_csv("../data/processed/modeling_table.csv", index=False)